In [1]:
import os
import time
import pickle
import argparse
import logging
import numpy as np
import pandas as pd
from collections import defaultdict

In [2]:
n_users = 0
n_items = 0
data = defaultdict(list)

In [3]:
# user, item, rating, timestamp

data = defaultdict(list)
i_stat = defaultdict(int)
cnt=0

with open('./ml-100k/u.data') as f:
    #next(f)
    for line in f:
        u, i, r, t = line.strip().split('\t')
        data[u].append((i, int(t)))
        i_stat[i] += 1
        #cnt+=1
    #%%
       # print('line is '+ str(cnt))

In [5]:
print('# user', len(data))
print('# item', len(i_stat))

behavior_len = [len(v) for k, v in data.items()]
total_num = sum(behavior_len)
print('# records', total_num)

max_u_len = np.max(behavior_len)
min_u_len = np.min(behavior_len)
avg_u_len = np.mean(behavior_len)
med_u_len = np.median(behavior_len)

print('max user behavior seq len: ', max_u_len)
print('min user behavior seq len: ', min_u_len)
print('avg user behavior seq len: ', avg_u_len)
print('median user behavior seq len: ', med_u_len)


i_u_len = [v for k, v in i_stat.items()]
max_i_len = np.max(i_u_len)
min_i_len = np.min(i_u_len)
avg_i_len = np.mean(i_u_len)
med_i_len = np.median(i_u_len)

print('max item behavior seq len: ', max_i_len)
print('min item behavior seq len: ', min_i_len)
print('avg item behavior seq len: ', avg_i_len)
print('median item behavior seq len: ', med_i_len)

print('!')
max_len_uid = 0
for k, v in data.items():
    if len(v) >= max_u_len:
        max_len_uid = k
        break

print('the max seq len user: ', max_len_uid)

valid_user_20 = {k:v for k, v in data.items() if len(v) > 20}
print('# users with len > 20: ', len(valid_user_20))

valid_user_40 = {k:v for k, v in data.items() if len(v) > 40}
print('# users with len > 40: ', len(valid_user_40))

valid_user_20_2k = {k:v for k, v in data.items() if len(v) > 20 and len(v) < 2000}
print('# users with 2000 > len > 20: ', len(valid_user_20_2k))

valid_user_40_2k = {k:v for k, v in data.items() if len(v) > 40 and len(v) < 2000}
print('# users with 2000 > len > 40: ', len(valid_user_40_2k))

valid_user_20_1k = {k:v for k, v in data.items() if len(v) > 20 and len(v) < 1000}
print('# users with 1000 > len > 20: ', len(valid_user_20_1k))

valid_user_40_1k = {k:v for k, v in data.items() if len(v) > 40 and len(v) < 1000}
print('# users with 1000 > len > 40: ', len(valid_user_40_1k))

valid_item_5 = {k:v for k, v in i_stat.items() if v > 5}
print('# item with freq > 5: ', len(valid_item_5))

print('# records with 1000 > seq len > 40', sum([len(v) for k, v in valid_user_40_1k.items()]))

print('# records with 2000 > seq len > 40', sum([len(v) for k, v in valid_user_40_2k.items()]))


# user 943
# item 1682
# records 100000
max user behavior seq len:  737
min user behavior seq len:  20
avg user behavior seq len:  106.04453870625663
median user behavior seq len:  65.0
max item behavior seq len:  583
min item behavior seq len:  1
avg item behavior seq len:  59.45303210463734
median item behavior seq len:  27.0
!
the max seq len user:  405
# users with len > 20:  911
# users with len > 40:  638
# users with 2000 > len > 20:  911
# users with 2000 > len > 40:  638
# users with 1000 > len > 20:  911
# users with 1000 > len > 40:  638
# item with freq > 5:  1298
# records with 1000 > seq len > 40 91610
# records with 2000 > seq len > 40 91610


# save data

In [5]:
u_vocab = {}
i_vocab = {}
for u, hist in data.items():
    if u not in u_vocab:
        u_vocab[u] = len(u_vocab) + 1
    for i, t in hist:
        if i not in i_vocab:
            i_vocab[i] = len(i_vocab) + 1
    hist = [(i_vocab[i], t) for i, t in hist]


import json
output = {
        'user_vocab': u_vocab,
        'item_vocab': i_vocab
        }

with open('./ml-100k/ml-100k_vocab.json2', 'w') as f:
    json.dump(output, f, indent=4)

max([v for k, v in u_vocab.items()]), max([v for k, v in i_vocab.items()])

data = {u_vocab[u]: [(i_vocab[i], t) for i, t in seq] for u, seq in data.items()}

# sorted by timestamp
data = {k:sorted(v, key=lambda x: x[1]) for k, v in data.items()}

# write file
with open(os.path.join('./ml-100k/', 'ml-100k2.csv'), 'w') as f:
    f.write('user_id\titem_id\ttime\n')
    for u, seq in data.items():
        for i, t in seq:
             f.write('\t'.join(map(str, [u, i, t]))+'\n')



len(u_vocab), len(i_vocab)

(943, 1682)

In [6]:
max([v for k, v in u_vocab.items()])

943

# find timestamp for analysis by data percentage

In [7]:
def build_init_data_by_ts(train, ratio):
    full_seq = []
    for u, seq in train.items():
        full_seq += seq

    full_seq = sorted(full_seq, key=lambda x: x[1])
    idx = int(len(full_seq) * ratio + 0.5)
    timestamp = full_seq[idx - 1][1]

    init_train = {}
    for u, seq in train.items():
        init_train[u] = [x for x in seq if x[1] < timestamp]
    
    return init_train, timestamp


def build_init_data_by_ts_and_k(train, ratio, init_k):
    total_num = sum([len(seq) for u, seq in train.items()])

    init_data = {u: seq[: init_k] for u, seq in train.items()}
    remain_data = {u: seq[init_k:] for u, seq in train.items()}

    idx = int(total_num * ratio + 0.5) - init_k * len(train)
    
    remain_seq = []
    for u, seq in remain_data.items():
        remain_seq += seq

    remain_seq = sorted(remain_seq, key=lambda x: x[1])
    timestamp = remain_seq[idx - 1][1]
    
    init_train = {}
    for u, seq in remain_data.items():
        init_train[u] = init_data[u] + [x for x in seq if x[1] < timestamp]

    return init_train, timestamp


def init_data_stat(init_train, train):
    init_item_set = set()
    init_empty_user = []
    for u, seq in init_train.items():
        init_item_set = init_item_set.union(set([x[0] for x in seq]))
        if len(seq) == 0:
            init_empty_user.append(u)

    behavior_len = [len(v) for k, v in init_train.items()]

    print('# init train :', sum(behavior_len))
    print('# data ratio :', 1.0 * sum(behavior_len) / sum([len(seq) for u, seq in train.items()]))
    print('# empty user in train number:', len(init_empty_user))

    max_u_len = np.max(behavior_len)
    min_u_len = np.min(behavior_len)
    avg_u_len = np.mean(behavior_len)
    med_u_len = np.median(behavior_len)
    print('max user behavior seq len: ', max_u_len)
    print('min user behavior seq len: ', min_u_len)
    print('avg user behavior seq len: ', avg_u_len)
    print('median user behavior seq len: ', med_u_len)     

    print('====================== test item coverage =====================')
    print('# items in init training data: ', len(init_item_set))  
    coverage = 0.
    lost_num = 0

    for u, x in test.items():
        if x[0] in init_item_set:
            coverage += 1
        else:
            lost_num += 1
    print('test coverage in train: ', coverage, coverage / (coverage + lost_num))
    print('# test items not found in training data: ', lost_num)


In [9]:
data = defaultdict(list)
with open(os.path.join('./ml-100k/', 'ml-100k.csv')) as f:
    next(f)
    for line in f:
        u, i, t = line.strip().split('\t')
        data[int(u)].append((int(i), int(t)))

user_clicked_set = {}
full_traim_num = 0
train, test = {}, {}
for u, seq in data.items():
    seq = sorted(seq, key=lambda x: x[1])
    train[u], test[u] = seq[:-1], seq[-1]
    user_clicked_set[u] = set([x[0] for x in train[u]])
    full_traim_num += len(train[u])

train_item_set = set()
for u, seq in train.items():
    train_item_set = train_item_set.union(set([x[0] for x in seq]))

coverage = 0.
lost_num = 0

for u, x in test.items():
    if x[0] in train_item_set:
        coverage += 1
    else:
        lost_num += 1
        
coverage /= len(test)
print('test coverage in train: ', coverage, coverage / (coverage + lost_num))
print('# test items not found in training data: ', lost_num)

test coverage in train:  0.9968186638388123 0.2494030246749801
# test items not found in training data:  3


In [10]:
print('# full train :', sum([len(seq) for u, seq in train.items()]))
    
# 30% init data
print('================ build data only using timestamp ================')
init_train, timestamp = build_init_data_by_ts(train, 0.3)
print('timestamp for 0.3 training data: ', timestamp)
init_data_stat(init_train, train)

print('\nbuild data using timestamp and make sure each user has at least 5 behaviors')
init_train, timestamp = build_init_data_by_ts_and_k(train, 0.3, 5)
print('timestamp for 0.3 and init 5 training data: ', timestamp)
init_data_stat(init_train, train)

# full train : 99057
================ build data only using timestamp ================
timestamp for 0.3 training data:  879744218
# init train : 29713
# data ratio : 0.29995860968937077
# empty user in train number: 611
max user behavior seq len:  483
min user behavior seq len:  0
avg user behavior seq len:  31.50901378579003
median user behavior seq len:  0.0
====================== test item coverage =====================
# items in init training data:  1388
test coverage in train:  844.0 0.8950159066808059
# test items not found in training data:  99

build data using timestamp and make sure each user has at least 5 behaviors
timestamp for 0.3 and init 5 training data:  879483941
# init train : 29716
# data ratio : 0.2999888952825141
# empty user in train number: 0
max user behavior seq len:  434
min user behavior seq len:  5
avg user behavior seq len:  31.51219512195122
median user behavior seq len:  5.0
====================== test item coverage =====================
# items in ini

In [11]:
print('# full train :', sum([len(seq) for u, seq in train.items()]))
    
# 50% init data
ratio = 0.5
print('================ build data only using timestamp ================')
init_train, timestamp = build_init_data_by_ts(train, ratio)
print('timestamp {} for {} training data: '.format(timestamp, ratio))
init_data_stat(init_train, train)

print('\nbuild data using timestamp and make sure each user has at least 5 behaviors')
init_train, timestamp = build_init_data_by_ts_and_k(train, ratio, 5)
print('timestamp {} for {} and init 5 training data: '.format(timestamp, ratio))
init_data_stat(init_train, train)

# full train : 99057
================ build data only using timestamp ================
timestamp 882823978 for 0.5 training data: 
# init train : 49526
# data ratio : 0.4999747620057139
# empty user in train number: 452
max user behavior seq len:  523
min user behavior seq len:  0
avg user behavior seq len:  52.51961823966066
median user behavior seq len:  19.0
====================== test item coverage =====================
# items in init training data:  1464
test coverage in train:  867.0 0.9194061505832449
# test items not found in training data:  76

build data using timestamp and make sure each user has at least 5 behaviors
timestamp 882388353 for 0.5 and init 5 training data: 
# init train : 49528
# data ratio : 0.4999949524011428
# empty user in train number: 0
max user behavior seq len:  483
min user behavior seq len:  5
avg user behavior seq len:  52.52173913043478
median user behavior seq len:  9.0
====================== test item coverage =====================
# items in ini

In [ ]:
print('# full train :', sum([len(seq) for u, seq in train.items()]))
    
# 70% init data
ratio = 0.7
print('================ build data only using timestamp ================')
init_train, timestamp = build_init_data_by_ts(train, ratio)
print('timestamp {} for {} training data: '.format(timestamp, ratio))
init_data_stat(init_train, train)

print('\nbuild data using timestamp and make sure each user has at least 5 behaviors')
init_train, timestamp = build_init_data_by_ts_and_k(train, ratio, 5)
print('timestamp {} for {} and init 5 training data: '.format(timestamp, ratio))
init_data_stat(init_train, train)

In [ ]:
print('# full train :', sum([len(seq) for u, seq in train.items()]))

# 10% init data
ratio = 0.1
print('================ build data only using timestamp ================')
init_train, timestamp = build_init_data_by_ts(train, ratio)
print('timestamp {} for {} training data: '.format(timestamp, ratio))
init_data_stat(init_train, train)

print('\nbuild data using timestamp and make sure each user has at least 5 behaviors')
init_train, timestamp = build_init_data_by_ts_and_k(train, ratio, 5)
print('timestamp {} for {} and init 5 training data: '.format(timestamp, ratio))
init_data_stat(init_train, train)